In [18]:
%pip install -r ~/code/GitHub/UPennContrast/devops/girder/annotation_client/requirements.txt
%pip install -e ~/code/GitHub/UPennContrast/devops/girder/annotation_client/
%pip install pandas
%pip install imageio
%pip install rtree
%pip install scikit-image
%pip install shapely
%pip install nd2reader
%pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///Users/arjunraj/code/GitHub/UPennContrast/devops/girder/annotation_client
  Attempting uninstall: annotation-client
    Found existing installation: annotation-client 0.1
    Uninstalling annotation-client-0.1:
      Successfully uninstalled annotation-client-0.1
  Running setup.py develop for annotation-client
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading 

In [19]:
import girder_client
import annotation_client.annotations as annotations
import annotation_client.tiles as tiles
import pandas as pd
import annotation_tools
#import imageio
from shapely.geometry import Point, Polygon, LineString
#pd.set_option('display.max_colwidth', None)
import numpy as np

In [2]:
apiUrl = 'http://localhost:8080/api/v1'
username = 'admin'
password = 'password'

In [3]:
client = girder_client.GirderClient(apiUrl=apiUrl)
user_id = client.authenticate(username=username, password=password)['_id']
annotationClient = annotations.UPennContrastAnnotationClient(apiUrl=apiUrl, token=client.token)

In [4]:
datasets = dict()
for folder in annotationClient.client.listFolder(user_id,'user'):
    for dataset in annotationClient.client.listFolder(folder['_id']):
        datasets[dataset['name']] = dataset['_id']

pd.DataFrame(datasets.keys(),columns=['Dataset Name'])

,Dataset Name
0,111111_20210723_155544_798__Plate20210723_STR_...
1,111_20200110_2Dent_001
2,20200110_2Dent_001_maxmerge
3,"20221122_143426_960__WellA4_ChannelDAPI,CY3,A5..."
4,"20230314_175616_594__WellB2_ChannelDAPI,YFP,CY..."
...,...
159,UNROLL_MS2_1_maxmerge
160,Well2_50framesCOMPOSITENORECODE
161,Well2_50framesCOMPOSITENORECODE_maxmerge
162,"WellA1_ChannelBrightfield,YFP,CY5,DAPI,A594_Se..."


In [5]:
dataset_name = '300cell-gastruloids_E11_1_04d16h00m'

In [6]:
datasetId = datasets[dataset_name]
datasetClient = tiles.UPennContrastDataset(apiUrl=apiUrl, token=client.token, datasetId=datasetId)
print(datasetClient.tiles)
tile_width = datasetClient.tiles['tileWidth']
tile_height = datasetClient.tiles['tileHeight']
print(tile_width)
print(tile_height)

{'bandCount': 1, 'dtype': 'uint8', 'levels': 3, 'magnification': 0.02834645669291339, 'mm_x': 0.35277777777777775, 'mm_y': 0.35277777777777775, 'sizeX': 2400, 'sizeY': 2400, 'tileHeight': 1024, 'tileWidth': 1024}
1024
1024


In [12]:
# May need to change the limit for large numbers of annotations. Default is 50.
pointAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='point')
blobAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='polygon')
lineAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, shape='line')
allAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId)

In [8]:
import pprint
print(datasetId)
connectionList = annotationClient.getAnnotationConnections(datasetId)
pprint.pprint(connectionList,indent=2)

64fa638546017e1063ec2941
[ { '_id': '64faf35446017e1063ec298d',
    'access': { 'groups': [],
                'users': [ { 'flags': [],
                             'id': '62891fc02060d362984bc104',
                             'level': 2}]},
    'childId': '64fa64ba46017e1063ec297b',
    'datasetId': '64fa638546017e1063ec2941',
    'parentId': '64fa63e446017e1063ec2969',
    'tags': ['yourPoint', 'myPoint']},
  { '_id': '64faf35446017e1063ec2990',
    'access': { 'groups': [],
                'users': [ { 'flags': [],
                             'id': '62891fc02060d362984bc104',
                             'level': 2}]},
    'childId': '64fa64bc46017e1063ec297e',
    'datasetId': '64fa638546017e1063ec2941',
    'parentId': '64fa63e546017e1063ec296c',
    'tags': ['yourPoint', 'myPoint']}]


In [13]:
print(len(blobAnnotationList))
print(len(pointAnnotationList))
print(len(allAnnotationList))
#print(len(lineAnnotationList))
print(blobAnnotationList[0])
print(pointAnnotationList[0])
#print(lineAnnotationList[0])

6
12
18
{'_id': '64fa63d946017e1063ec2957', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 0, 'coordinates': [{'x': 431.2422919282002, 'y': 921.8210984152914}, {'x': 415.66523156277253, 'y': 887.5515656113505}, {'x': 396.97275912425937, 'y': 871.9745052459228}, {'x': 319.0874572971211, 'y': 887.5515656113505}, {'x': 319.0874572971211, 'y': 974.7831036577454}, {'x': 337.77992973563425, 'y': 990.3601640231731}, {'x': 362.70322632031855, 'y': 993.4755760962586}, {'x': 390.74193497808835, 'y': 993.4755760962586}, {'x': 418.78064363585804, 'y': 981.0139278039164}, {'x': 434.3577040012857, 'y': 956.0906312192323}], 'datasetId': '64fa638546017e1063ec2941', 'location': {'Time': 0, 'XY': 0, 'Z': 0}, 'shape': 'polygon', 'tags': ['myBlob']}
{'_id': '64fa63e446017e1063ec2969', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 0, 'coordinates': [{'x': 334.66451766254875, 'y': 794

In [14]:
pointList1 = annotation_tools.get_annotations_with_tags(allAnnotationList,['myPoint'],exclusive=False)
pointList2 = annotation_tools.get_annotations_with_tags(allAnnotationList,['yourPoint'],exclusive=False)
blobList1 = annotation_tools.get_annotations_with_tags(allAnnotationList,['myBlob'],exclusive=False)
print(len(pointList1))
print(len(pointList2))
print(len(blobList1))

6
6
6


In [21]:
parentTag = 'myBlob'
childTag = 'myPoint'

# Create connections between all points and blobs with the same tags
parentList = annotation_tools.get_annotations_with_tags(blobAnnotationList,[parentTag],exclusive=False)
childList = annotation_tools.get_annotations_with_tags(pointAnnotationList,[childTag],exclusive=False)

print(len(parentList))
print(len(childList))
print(parentList)
print(childList)

6
6
[{'_id': '64fa63d946017e1063ec2957', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 0, 'coordinates': [{'x': 431.2422919282002, 'y': 921.8210984152914}, {'x': 415.66523156277253, 'y': 887.5515656113505}, {'x': 396.97275912425937, 'y': 871.9745052459228}, {'x': 319.0874572971211, 'y': 887.5515656113505}, {'x': 319.0874572971211, 'y': 974.7831036577454}, {'x': 337.77992973563425, 'y': 990.3601640231731}, {'x': 362.70322632031855, 'y': 993.4755760962586}, {'x': 390.74193497808835, 'y': 993.4755760962586}, {'x': 418.78064363585804, 'y': 981.0139278039164}, {'x': 434.3577040012857, 'y': 956.0906312192323}], 'datasetId': '64fa638546017e1063ec2941', 'location': {'Time': 0, 'XY': 0, 'Z': 0}, 'shape': 'polygon', 'tags': ['myBlob']}, {'_id': '64fa63da46017e1063ec295a', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 0, 'coordinates': [{'x': 1138.4408325186157, 'y': 713.0

In [27]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from scipy.spatial import cKDTree
import numpy as np

# Example blob list (trimmed for brevity here)
blob_list = parentList  # Your provided blob list goes here

# Extract polygons from blob list
polygon_data = [{'polygon_id': blob['_id'], 'polygon_geom': Polygon([(pt['x'], pt['y']) for pt in blob['coordinates']])} for blob in blob_list]
poly_df = pd.DataFrame(polygon_data)
gdf_poly = gpd.GeoDataFrame(poly_df, geometry='polygon_geom')



# Extract points from childList
point_data = [{'point_id': child['_id'], 'x': child['coordinates'][0]['x'], 'y': child['coordinates'][0]['y']} for child in childList]
point_df = pd.DataFrame(point_data)
pprint.pprint(point_df)
gdf_point = gpd.GeoDataFrame(point_df, geometry=gpd.points_from_xy(point_df.x, point_df.y))

# Create cKDTree object to represent polygon centroids
tree = cKDTree(np.array(list(zip(gdf_poly.geometry.centroid.x, gdf_poly.geometry.centroid.y))))

# Compute distance and index for the nearest polygon for each point
distances, indices = tree.query(np.array(list(zip(gdf_point.geometry.x, gdf_point.geometry.y))))

# Map point IDs to nearest polygon IDs
point_to_polygon = pd.DataFrame({
    'point_id': point_df['point_id'],
    'nearest_polygon_id': gdf_poly.iloc[indices]['polygon_id'].values
})

print(point_to_polygon)



Empty DataFrame
Columns: []
Index: []


KeyError: 'x'

In [20]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from scipy.spatial import cKDTree
import numpy as np

In [26]:
myNewConnections = []
annotation1 = pointList1[0]
annotation2 = pointList2[0]
combined_tags = list(set(annotation1['tags'] + annotation2['tags']))
myNewConnections.append({'datasetId': datasetId,'parentId': annotation1['_id'], 'childId': annotation2['_id'],'tags': combined_tags})

annotation1 = pointList1[1]
annotation2 = pointList2[1]
combined_tags = list(set(annotation1['tags'] + annotation2['tags']))
myNewConnections.append({'datasetId': datasetId,'parentId': annotation1['_id'], 'childId': annotation2['_id'],'tags': combined_tags})


pprint.pprint(myNewConnections,indent=2)
for connection in myNewConnections:
    annotationClient.createConnection(connection)


[ { 'childId': '64fa64ba46017e1063ec297b',
    'datasetId': '64fa638546017e1063ec2941',
    'parentId': '64fa63e446017e1063ec2969',
    'tags': ['yourPoint', 'myPoint']},
  { 'childId': '64fa64bc46017e1063ec297e',
    'datasetId': '64fa638546017e1063ec2941',
    'parentId': '64fa63e546017e1063ec296c',
    'tags': ['yourPoint', 'myPoint']}]


In [24]:
annotation1 = filteredPointList[0].copy()
annotation2 = targetPointList[0].copy()
#print(datasetId)
#pprint.pprint(annotation1)
#pprint.pprint(annotation2)

combined_tags = list(set(annotation1['tags'] + annotation2['tags']))
print(combined_tags)

myNewConnection = {'datasetId': datasetId,'parentId': annotation1['_id'], 'childId': annotation2['_id'],'tags': combined_tags}
pprint.pprint(myNewConnection)

#annotationClient.client.post(PATHS['connection'], json=myNewConnection)
annotationClient.createConnection(myNewConnection)


['outerPt', 'innerPt']
{'childId': '64aa99e3f69bc320e7503856',
 'datasetId': '64aa2973f69bc320e7503648',
 'parentId': '64aa9970f69bc320e7503830',
 'tags': ['outerPt', 'innerPt']}


{'_id': '64b66ca75a47f8266db0f1cf',
 'access': {'groups': [],
  'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]},
 'childId': '64aa99e3f69bc320e7503856',
 'datasetId': '64aa2973f69bc320e7503648',
 'parentId': '64aa9970f69bc320e7503830',
 'tags': ['outerPt', 'innerPt']}

In [10]:
import math

def calculate_distance(point1, point2):
    x1, y1, z1 = point1['coordinates'][0]['x'], point1['coordinates'][0]['y'], point1['coordinates'][0]['z']
    x2, y2, z2 = point2['coordinates'][0]['x'], point2['coordinates'][0]['y'], point2['coordinates'][0]['z']
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)


def find_matching_annotations_by_location(source, target_list, Time=True, XY=True, Z=True):
    """
    This function filters the target_list based on the 'location' of the source point.
    The function parameters 'Time', 'XY', and 'Z' can be set to True or False to specify whether these 'location' attributes need to be matched.
    By default, all of these parameters are set to True, meaning all 'location' attributes need to match.

    Parameters:
    source (dict): The source point annotation object
    target_list (list): The list of target point annotation objects
    Time (bool): Specifies whether the 'Time' attribute of 'location' needs to be matched. Default is True.
    XY (bool): Specifies whether the 'XY' attribute of 'location' needs to be matched. Default is True.
    Z (bool): Specifies whether the 'Z' attribute of 'location' needs to be matched. Default is True.

    Returns:
    list: The filtered list of target point annotation objects that match the specified 'location' attributes

    Example of usage:
    1) Matching all 'location' attributes:
    source = {...}  # source point annotation object
    target_list = [...]  # target point annotation list
    matching_annotations = find_matching_annotations_by_location(source, target_list)

    2) Matching specified 'location' attributes (in this case, 'Time' and 'XY'):
    source = {...}  # source point annotation object
    target_list = [...]  # target point annotation list
    matching_annotations = find_matching_annotations_by_location(source, target_list, Time=True, XY=True, Z=False)
    """
    params = {'Time': Time, 'XY': XY, 'Z': Z}
    return [target for target in target_list if all(source['location'].get(attr) == target['location'].get(attr) for attr, value in params.items() if value)]

def find_closest_point(source, target_list):
    min_distance = float('inf')
    closest_point = None
    for target in target_list:
        if source['_id'] == target['_id']:
            continue
        distance = calculate_distance(source, target)
        if distance < min_distance:
            min_distance = distance
            closest_point = target
    return closest_point, min_distance

#filteredPointList = [...]  # Your filteredPointList here
#targetPointList = [...]  # Your targetPointList here

for source in filteredPointList:
    # Add the attributes you want to match as keyword arguments here. If no arguments are provided, all 'location' attributes will be matched.
    matching_annotations = find_matching_annotations_by_location(source, targetPointList, Time=True, XY=True)
    closest_point, min_distance = find_closest_point(source, matching_annotations)
    print(f"For point id {source['_id']}, closest point is {closest_point['_id']} with distance {min_distance}")

for source in filteredPointList:
    # Add the attributes you want to match as keyword arguments here. If no arguments are provided, all 'location' attributes will be matched.
    matching_annotations = find_matching_annotations_by_location(source, targetPointList)
    closest_point, min_distance = find_closest_point(source, matching_annotations)
    print(f"For point id {source['_id']}, closest point is {closest_point['_id']} with distance {min_distance}")



For point id 64aa9970f69bc320e7503830, closest point is 64aa99e4f69bc320e7503858 with distance 44.97019871692391
For point id 64aa9971f69bc320e7503832, closest point is 64aa99e4f69bc320e7503858 with distance 40.542745732502006
For point id 64aa9972f69bc320e7503834, closest point is 64aa99ecf69bc320e7503864 with distance 22.147569722466812
For point id 64aa9972f69bc320e7503836, closest point is 64aa99e3f69bc320e7503856 with distance 23.26450919111039
For point id 64aa9973f69bc320e7503838, closest point is 64aa99e4f69bc320e7503858 with distance 15.882769465319464
For point id 64aa9974f69bc320e750383a, closest point is 64aa99e4f69bc320e7503858 with distance 36.323228159944755
For point id 64aa9990f69bc320e7503852, closest point is 64aa9a46f69bc320e750386c with distance 7.65561016396188


TypeError: 'NoneType' object is not subscriptable